In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torchsummary import summary

In [ ]:
class SIGNET(nn.Module):
    def __init__(self, input_size):
        super(SIGNET, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=input_size, out_channels=96, kernel_size=(11, 11), stride=(1, 1))
        self.relu = nn.ReLU()
        self.LRN = nn.LocalResponseNorm(size=5, alpha=1e-4, beta=0.75, k=2.0)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)

        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=(5, 5), stride=(1, 1), padding=2)
        self.Drpout = nn.Dropout(p=0.3)

        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=(3, 3), stride=(1, 1), padding=1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=(3, 3), stride=(1, 1), padding=1)


        self.fc1 = nn.Linear(in_features=108800, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=128)


    def forward(self, x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.LRN(out)
        out = self.maxpool(out)

        out = self.conv2(out)
        out = self.relu(out)
        out = self.LRN(out)
        out = self.maxpool(out)
        out = self.Drpout(out)

        out = self.conv3(out)
        out = self.relu(out)
        out = self.conv4(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.Drpout(out)

        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.Drpout(out)
        out = self.fc2(out)

        return out


In [ ]:
# print model
model = SIGNET(input_size=1)
print(model)

SIGNET(
  (conv1): Conv2d(1, 96, kernel_size=(11, 11), stride=(1, 1))
  (relu): ReLU()
  (LRN): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2.0)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (Drpout): Dropout(p=0.3, inplace=False)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=95744, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
model = SIGNET(input_size=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)  # Move the model to the GPU

summary(model, input_size=(1, 155, 220))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 145, 210]          11,712
              ReLU-2         [-1, 96, 145, 210]               0
 LocalResponseNorm-3         [-1, 96, 145, 210]               0
         MaxPool2d-4          [-1, 96, 72, 104]               0
            Conv2d-5         [-1, 256, 72, 104]         614,656
              ReLU-6         [-1, 256, 72, 104]               0
 LocalResponseNorm-7         [-1, 256, 72, 104]               0
         MaxPool2d-8          [-1, 256, 35, 51]               0
           Dropout-9          [-1, 256, 35, 51]               0
           Conv2d-10          [-1, 384, 35, 51]         885,120
             ReLU-11          [-1, 384, 35, 51]               0
           Conv2d-12          [-1, 256, 35, 51]         884,992
             ReLU-13          [-1, 256, 35, 51]               0
        MaxPool2d-14          [-1, 256,

In [ ]:
class SIGNET(nn.Module):
    def __init__(self, input_size):
        super(SIGNET, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=input_size, out_channels=96, kernel_size=(11, 11), stride=(1, 1)),
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=1e-4, beta=0.75, k=2.0),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=(5, 5), stride=(1,1)),
nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=1e-4, beta=0.75, k=2.0),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=0.3)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.ReLU(),
)
        self.layer4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=0.3)
        )
        self.layer5 = nn.Sequential(
            nn.Linear(in_features=98304, out_features=1024),
            nn.Dropout(p=0.5)
        )
        self.layer6 = nn.Sequential(
            nn.Linear(in_features=1024, out_features=128)
        )



    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1)
        out = self.layer5(out)
        out = self.layer6(out)

        return out

In [ ]:
# print model
model = SIGNET(input_size=1)
print(model)

SIGNET(
  (layer1): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(1, 1))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2.0)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2.0)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3, inplace=False)
  )
  (layer3): Sequential(
    (0): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (layer4): Sequential(
    (0): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): Dropout(p=0.3, inplace=False)
  )
  (layer5): Sequential(
    (0): Linear(in_features=86016, out_features=1024, bias=True)

In [ ]:
model = SIGNET(input_size=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)  # Move the model to the GPU

summary(model, input_size=(1, 155, 220))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 145, 210]          11,712
              ReLU-2         [-1, 96, 145, 210]               0
 LocalResponseNorm-3         [-1, 96, 145, 210]               0
         MaxPool2d-4          [-1, 96, 72, 104]               0
            Conv2d-5         [-1, 256, 68, 100]         614,656
              ReLU-6         [-1, 256, 68, 100]               0
 LocalResponseNorm-7         [-1, 256, 68, 100]               0
         MaxPool2d-8          [-1, 256, 33, 49]               0
           Dropout-9          [-1, 256, 33, 49]               0
           Conv2d-10          [-1, 384, 33, 49]         885,120
             ReLU-11          [-1, 384, 33, 49]               0
           Conv2d-12          [-1, 256, 33, 49]         884,992
             ReLU-13          [-1, 256, 33, 49]               0
        MaxPool2d-14          [-1, 256,